## imports

In [1]:
from os import environ as ENV
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from pymssql import connect
import pandas as pd
from boto3 import client

## load data in

In [2]:
load_dotenv()

True

In [3]:
def get_db_connection(config: dict) -> connect:
    """Returns database connection."""

    return connect(
        server=config["DB_HOST"],
        port=config["DB_PORT"],
        user=config["DB_USER"],
        database=config["DB_NAME"],
        password=config["DB_PASSWORD"],
        as_dict=True
    )

In [4]:
connection = get_db_connection(ENV)

In [5]:
def get_df(conn: connect) -> pd.DataFrame:
    """Returns a Dataframe of method data from database."""

    query = """ 
            SELECT *
            FROM s_beta.recording AS r
            FULL JOIN s_beta.plant AS p
                ON r.plant_id = p.plant_id
            """
    
    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()

    df = pd.DataFrame(rows)[
        ["recording_taken", "plant_id", "plant_name", "scientific_name", "soil_moisture", "temperature"]]
    
    return df

In [6]:
data = get_df(connection)

In [7]:
data = data.astype({"soil_moisture":"float64",
                    "temperature":"float64"})

In [8]:
data['recording_taken'] = pd.to_datetime(data['recording_taken'], utc=True)

In [9]:
data.dtypes

recording_taken    datetime64[ns, UTC]
plant_id                         int64
plant_name                      object
scientific_name                 object
soil_moisture                  float64
temperature                    float64
dtype: object

In [10]:
data

,recording_taken,plant_id,plant_name,scientific_name,soil_moisture,temperature
0,2024-04-17 09:44:01+00:00,0,Epipremnum Aureum,Epipremnum aureum,31.4063,13.1995
1,2024-04-17 09:44:01+00:00,1,Venus flytrap,None,29.7548,12.0470
2,2024-04-17 09:44:00+00:00,2,Corpse flower,None,35.9540,9.1711
3,2024-04-17 09:44:04+00:00,3,Rafflesia arnoldii,None,34.6505,10.0300
4,2024-04-17 09:44:10+00:00,4,Black bat flower,None,26.7741,11.3474
...,...,...,...,...,...,...
2433,2024-04-17 10:52:39+00:00,40,Amaryllis,Hippeastrum (group),30.8488,9.1521
2434,2024-04-17 10:52:40+00:00,41,Caladium Bicolor,Caladium bicolor,29.0599,11.8277
2435,2024-04-17 10:52:39+00:00,42,Chlorophytum Comosum,Chlorophytum comosum 'Vittatum',30.3319,9.9005
2436,2024-04-17 10:52:40+00:00,44,Araucaria Heterophylla,Araucaria heterophylla,26.7869,9.6246


## transform data

### summary

In [11]:
def get_summary(df: pd.DataFrame) -> pd.DataFrame:
    """Gets 1 mean per parameter per plant.
    Returns pd.DF."""
    
    df = df.drop(columns=["recording_taken"])
    
    df = df.groupby(["plant_id", "plant_name", "scientific_name"],as_index=False
                    ).agg(["mean", "std", "min", "max"]
                          ).droplevel(1, axis=1)
    
    df.columns = ['plant_id', 'plant_name', 'scientific_name'] + \
        [param+"_"+metric
         for param in ['soil_moisture', 'temperature']
         for metric in ["mean", "std", "min", "max"]]
    
    return df

In [12]:
summary = get_summary(data)

In [13]:
summary

,plant_id,plant_name,scientific_name,soil_moisture_mean,soil_moisture_std,soil_moisture_min,soil_moisture_max,temperature_mean,temperature_std,temperature_min,temperature_max
0,0,Epipremnum Aureum,Epipremnum aureum,28.945992,0.943194,27.4323,31.4063,13.618884,1.915160,13.1330,24.7830
1,5,Pitcher plant,Sarracenia catesbaei,33.571975,0.913545,32.1019,35.9741,11.191940,0.025514,11.1562,11.2455
2,6,Wollemi pine,Wollemia nobilis,30.444053,0.903123,28.9930,32.7930,12.886500,11.879152,10.9444,94.9506
3,8,Bird of paradise,Heliconia schiedeana 'Fire and Ice',24.675626,0.962709,23.0963,27.1754,11.530380,0.034152,11.4838,11.6025
4,9,Cactus,Pereskia grandifolia,29.623714,0.940831,28.1315,32.0963,10.733118,0.936536,10.5765,17.2890
5,11,Asclepias Curassavica,Asclepias curassavica,28.031373,12.845303,-62.5927,32.2118,11.075952,9.552786,9.4339,77.8178
6,14,Colocasia Esculenta,Colocasia esculenta,31.766929,0.914958,30.3162,34.1746,15.460525,10.769446,12.7846,75.9186
7,16,Euphorbia Cotinifolia,Euphorbia cotinifolia,-5.895437,1.364163,-8.1644,-2.2859,13.339345,0.447712,12.6945,16.4157
8,17,Ipomoea Batatas,Ipomoea batatas,28.776578,0.935278,27.2677,31.2443,9.922176,0.995606,9.7386,16.8184
9,19,Musa Basjoo,Musa basjoo,27.550073,13.221675,-64.8078,31.7957,15.712404,11.436185,13.8426,94.6848


### anomalies

In [14]:
anomalies = data.copy(deep=True)

In [15]:
def get_std(row: dict, df: pd.DataFrame, col: str) -> int:
    """Compare minutely value to mean of past hour;
    Returns std."""
    
    last_hour = pd.Timestamp(datetime.now(timezone.utc)-timedelta(hours=1))
    last_hour_vals = df[(df["plant_id"] == row["plant_id"]) &
                        (df["recording_taken"] >= last_hour)][col]
    
    mean = last_hour_vals.mean()
    std = last_hour_vals.std()
    
    nstd = (row[col] - mean) / std
    
    return nstd

In [16]:
anomalies["soil_moisture_nstd"] = data.apply(get_std,
                                             args=(data, "soil_moisture"),
                                             axis=1)

In [17]:
anomalies["temperature_nstd"] = data.apply(get_std,
                                           args=(data, "temperature"),
                                           axis=1)

In [18]:
anomalies = anomalies[(anomalies["soil_moisture_nstd"] <= -2.5) |
                      (anomalies["soil_moisture_nstd"] >= 2.5) |
                      (anomalies["temperature_nstd"] <= -2.5) |
                      (anomalies["temperature_nstd"] >= 2.5)]

In [19]:
anomalies

,recording_taken,plant_id,plant_name,scientific_name,soil_moisture,temperature,soil_moisture_nstd,temperature_nstd
0,2024-04-17 09:44:01+00:00,0,Epipremnum Aureum,Epipremnum aureum,31.4063,13.1995,2.838254,-0.221224
1,2024-04-17 09:44:01+00:00,1,Venus flytrap,None,29.7548,12.0470,2.873607,-0.132865
2,2024-04-17 09:44:00+00:00,2,Corpse flower,None,35.9540,9.1711,2.861548,-0.184705
3,2024-04-17 09:44:04+00:00,3,Rafflesia arnoldii,None,34.6505,10.0300,2.967427,-0.140707
4,2024-04-17 09:44:10+00:00,4,Black bat flower,None,26.7741,11.3474,3.005226,-0.121682
...,...,...,...,...,...,...,...,...
2086,2024-04-17 10:45:24+00:00,21,Anthurium,Anthurium andraeanum,30.4028,22.3906,-1.190810,4.597259
2107,2024-04-17 10:45:36+00:00,42,Chlorophytum Comosum,Chlorophytum comosum 'Vittatum',30.7394,27.7771,-1.169360,6.848287
2153,2024-04-17 10:46:40+00:00,40,Amaryllis,Hippeastrum (group),31.1805,14.6846,-1.266221,6.995321
2266,2024-04-17 10:49:35+00:00,14,Colocasia Esculenta,Colocasia esculenta,30.4780,58.0129,-1.448687,3.909097


## load data

### save to CSVs

In [20]:
summary.to_csv("summary.csv", index=False)

In [21]:
anomalies.to_csv("anomalies.csv", index=False)

### upload to S3

In [22]:
S3 = client('s3',
            aws_access_key_id=ENV["AWS_ACCESS_KEY_ID"],
            aws_secret_access_key=ENV["AWS_SECRET_ACCESS_KEY"])

In [23]:
def upload_object(client: client,
                  file: str,
                  key: str,
                  bucket: str = "late-ordovician") -> None:
    """Upload file to S3 bucket.
    Returns nothing."""
    
    client.upload_file(file, bucket, key)

In [24]:
upload_object(S3, "summary.csv", "summary.csv")

In [25]:
upload_object(S3, "anomalies.csv", "anomalies.csv")

## clear database